<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Цель-исследования" data-toc-modified-id="Цель-исследования-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Цель исследования</a></span></li><li><span><a href="#Задачи" data-toc-modified-id="Задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задачи</a></span></li><li><span><a href="#Материалы" data-toc-modified-id="Материалы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Материалы</a></span></li><li><span><a href="#Подключение-к-базе-данных" data-toc-modified-id="Подключение-к-базе-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Подключение к базе данных</a></span></li><li><span><a href="#Обзор-доступных-таблиц" data-toc-modified-id="Обзор-доступных-таблиц-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обзор доступных таблиц</a></span><ul class="toc-item"><li><span><a href="#books" data-toc-modified-id="books-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>books</a></span></li><li><span><a href="#authors" data-toc-modified-id="authors-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>authors</a></span></li><li><span><a href="#publishers" data-toc-modified-id="publishers-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>publishers</a></span></li><li><span><a href="#ratings" data-toc-modified-id="ratings-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>ratings</a></span></li><li><span><a href="#reviews" data-toc-modified-id="reviews-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>reviews</a></span></li></ul></li><li><span><a href="#Определение-искомых-значений" data-toc-modified-id="Определение-искомых-значений-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Определение искомых значений</a></span><ul class="toc-item"><li><span><a href="#Книги,-вышедшие-после-1-января-2000-года" data-toc-modified-id="Книги,-вышедшие-после-1-января-2000-года-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Книги, вышедшие после 1 января 2000 года</a></span></li><li><span><a href="#Число-обзоров-и-средняя-оценка-каждой-книги" data-toc-modified-id="Число-обзоров-и-средняя-оценка-каждой-книги-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Число обзоров и средняя оценка каждой книги</a></span></li><li><span><a href="#Издательство,-выпустившее-наибольшее-число-книг,-кроме-брошюр" data-toc-modified-id="Издательство,-выпустившее-наибольшее-число-книг,-кроме-брошюр-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Издательство, выпустившее наибольшее число книг, кроме брошюр</a></span></li><li><span><a href="#Автор-с-самой-высокой-средней-оценкой-книг" data-toc-modified-id="Автор-с-самой-высокой-средней-оценкой-книг-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Автор с самой высокой средней оценкой книг</a></span></li><li><span><a href="#Среднее-количество-обзоров-от-пользователей" data-toc-modified-id="Среднее-количество-обзоров-от-пользователей-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Среднее количество обзоров от пользователей</a></span></li></ul></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

# Исследование базы данных сервиса для чтения книг по подписке
## Цель исследования

Определить направление для формулировки преимуществ нового продукта на основе данных о вышедших книгах и их рейтингах, полученных от пользователей сервиса.

## Задачи

1. Подключение к базе данных.
1. Обзор доступных таблиц.
1. Определение ряда значений, а именно:
    - число книг, вышедших после 2000 года;
    - число обзоров для каждой книги и ее средняя оценка;
    - вывод из анализа книг, чей объем менее 50 страниц;
    - автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
    - среднее число обзоров от пользователей, которые поставили более 48 оценок.
4. Заключение.

## Материалы
[Google Colab](https://colab.research.google.com/drive/11NjLnkwI7y5v3z7RekjfDBbQ6f5uH9pi?usp=sharing "Презентация")

## Подключение к базе данных

In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Обзор доступных таблиц
### books

In [3]:
# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT COUNT(*) FROM books'''
get_sql_data(query)

,count
0,1000


### authors

In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT COUNT(*) FROM authors'''
get_sql_data(query)

,count
0,636


### publishers

In [7]:
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query = '''SELECT COUNT(*) FROM publishers'''
get_sql_data(query)

,count
0,340


### ratings

In [9]:
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''SELECT COUNT(*) FROM ratings'''
get_sql_data(query)

,count
0,6456


### reviews

In [11]:
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
query = '''SELECT COUNT(*) FROM reviews'''
get_sql_data(query)

,count
0,2793


## Определение искомых значений
### Книги, вышедшие после 1 января 2000 года

In [13]:
query = '''SELECT COUNT(*) FROM books WHERE publication_date >= '2000-01-01' '''
get_sql_data(query)

,count
0,821


После 1 января 2000 года была выпущена 821 книга.

### Число обзоров и средняя оценка каждой книги

In [14]:
query = '''with sel_rev as (select b.title, count(re.review_id) as rev_cnt
           from books as b
           join reviews as re on b.book_id = re.book_id
           group by b.title),
           sel_rat as (select b.title, avg(r.rating) as rat_avg
           from books as b
           join ratings as r on b.book_id = r.book_id
           group by b.title)
           select sel_rev.title, sel_rev.rev_cnt, sel_rat.rat_avg
           from sel_rev
           join sel_rat on sel_rev.title = sel_rat.title
           '''
get_sql_data(query)

,title,rev_cnt,rat_avg
0,The Count of Monte Cristo,5,4.217391
1,Count Zero (Sprawl #2),2,2.500000
2,The Botany of Desire: A Plant's-Eye View of th...,2,3.500000
3,The Poisonwood Bible,5,4.363636
4,The Canterbury Tales,3,3.333333
...,...,...,...
988,Of Love and Other Demons,2,4.500000
989,In the Heart of the Sea: The Tragedy of the Wh...,3,3.333333
990,Welcome to Temptation (Dempseys #1),2,5.000000
991,World's End (The Sandman #8),2,4.500000


Перед нами численность отзывов и средний рейтинг для каждой книги.

Благодарю за совместную работу и хочу поздравить, ты большая молодец, что прошла такой путь и освоила программу, надеюсь у тебя сбудутся все намеченные планы.

У Практикума развивается сообщество выпускников, так что рекомендую про него не забывать, в случае необходимости.


Ещё раз порекламирую функцию для первичного анализа, очень удобно автоматизироват ьповторяющиеся действия, а ещё есть библиотека `pandas profiling`, которая как раз позволяет автоматизировать первичный анализ. Для твоих визуализаций рекомендую набор фокусов ниже, они позволят сделать визуализации ещё информативнее. Понравилась твоя вовлечённость спасибо за многочисленные комментарии.  С `SQL`, такая маленькая описка, но рекомендую потренироваться в приоритете именно с `SQL`, если планируешь искать работу аналитиком, так как этот навык чаще всего проверяется. Удачи!) 

Оформление презентаций дело субъективное и зачастую в разных компаниях имеются свои требования, но вот видео от Яндекса по созданию презентаций https://www.youtube.com/watch?v=S0r0fMJa9eA, а ещё на платформе практикума открылся бесплатный курс по созданию презентаций, рекомендую.

Для более глубокого изучения tableau, в случае необходимости рекомендую материал https://tableau.pro/m01, этот курс делает классная команда, у них бывают и другие интересные мини курсы. Но в последнее время российские компании уходят от tableau или уже ушли, поэтому возможно придётся познакомиться с другими инструментами.

Задачи на статистику или вокруг неё часто возникают на собеседовании, для укрепления этого фундамента рекомендую статью для выбора критерия проверки гипотез https://lit-review.ru/biostatistika/vybor-statisticheskogo-kriteriya/, https://statpsy.ru/ - информация по критериям проверки гипотез.

Визуализации данных для аналитиков очень важны, при помощи них легче доносить желаемые мысли, а для того, чтобы лучше отобразить, нужно понимать, какие вообще существуют визуализации, делюсь с тобой своей подборкой для разных случае жизни https://www.notion.so/d9392e1cb71f48cfb1fbf7f557a2b7e7 

SQL важный навык, который нужно поддерживать для этого существует много тренажёров:
* Статья на хабр с топ-тренажёрами: https://techrocks.ru/2019/11/22/7-sites-to-perfect-sql-query-writing-skills/
* задачки SQL - https://habr.com/ru/company/dcmiran/blog/500360/
* https://sqlzoo.net/

### Издательство, выпустившее наибольшее число книг, кроме брошюр

In [15]:
query = '''WITH sel1 AS (SELECT p.publisher, b.book_id
                 FROM publishers AS p
                 JOIN books AS b ON p.publisher_id = b.publisher_id
                 WHERE b.num_pages > 50)
           SELECT publisher, COUNT(book_id) AS books_cnt
           FROM sel1
           GROUP BY publisher
           ORDER BY books_cnt DESC
           LIMIT 1
        '''
get_sql_data(query)

,publisher,books_cnt
0,Penguin Books,42


Самым продуктивным издательством стало Penguin Books с 42 книгами объемом более 50 страниц.

### Автор с самой высокой средней оценкой книг

In [16]:
query = '''
           WITH sel1 AS (SELECT b.book_id, COUNT(r.rating_id) AS rating_cnt
           FROM books as b
           JOIN ratings as r ON b.book_id = r.book_id
           GROUP BY b.book_id
           HAVING COUNT(r.rating_id) >= 50)
           SELECT a.author, AVG(r.rating)
           FROM authors AS a
           JOIN books as b ON a.author_id = b.author_id
           JOIN ratings as r ON b.book_id = r.book_id
           WHERE b.book_id IN (SELECT book_id FROM sel1)
           GROUP BY a.author
           ORDER BY AVG(r.rating) DESC
           LIMIT 1
           
        '''
get_sql_data(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Самую высокую среднюю оценку книг с учетом изданий, имеющих 50 и более оценок, получила Дж.К. Роулинг совместно с иллюстратором Мэри Грен-Прей.

### Среднее количество обзоров от пользователей

In [17]:
query = '''
           WITH sel1 AS (SELECT username, COUNT(rating_id) AS rating_cnt
           FROM ratings
           GROUP BY username
           HAVING COUNT(rating_id) > 48),
           sel2 AS (SELECT username, COUNT(review_id) AS review_cnt
           FROM reviews
           WHERE username IN (SELECT username FROM sel1)
           GROUP BY username)
           SELECT AVG(review_cnt)
           FROM sel2
        '''
get_sql_data(query)

,avg
0,24.0


Каждый пользователь, поставивший больше 48 оценок, в среднем написал 24 обзора.

## Заключение

Для формирования ценностного предложения мы можем предложить отделу маркетинга следующие направления:
1. Свыше 800 произведений современной и классической литературы.
1. Акции на самые популярные книги. Например, при покупке подписки на год, подарочное печатное издание книги из числа самых популярных по числу отзывов и по среднему рейтингу на выбор.
1. Новинки издательства Penguin Books доступны подписчикам раньше, чем в других сервисах или магазинах.
1. Обзоры и рейтинги от других читателей.